In [ ]:
import logging

import ax
import pandas as pd
import ray
from ax.service.ax_client import AxClient
from ax.plot.contour import interact_contour, plot_contour
from ax.plot.slice import plot_slice
from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.service.utils.best_point import get_best_from_model_predictions, get_best_raw_objective_point
from ray import tune
from ray.tune import track, JupyterNotebookReporter
from ray.tune.suggest.ax import AxSearch
from ray.tune.suggest.suggestion import SuggestionAlgorithm
from ax.modelbridge.generation_strategy import GenerationStep, GenerationStrategy
from ax.modelbridge.registry import Models

from synthesized import HighDimSynthesizer


root_logger = logging.getLogger()
root_logger.setLevel(50)
logger = logging.getLogger(__name__)


def train_evaluate(parameterization):
    with HighDimSynthesizer(df=data, **parameterization) as synthesizer:     
        synthesizer.learn(data, num_iterations=None)
        
        data_ = synthesizer.preprocess(data.sample(loss_sample_size))
        feed_dict = synthesizer.get_data_feed_dict(data_)
        losses = synthesizer.get_losses(data=feed_dict)
        
        loss = losses['total-loss'].numpy().item()

        track.log(
            mean_loss=loss,
            reconstruction_loss = losses['reconstruction-loss'].numpy().item(),
            kl_loss = losses['kl-loss'].numpy().item()
        )

class AxSearch2(SuggestionAlgorithm):
    """A wrapper around Ax to provide trial suggestions."""

    def __init__(self, ax_client, max_concurrent=10, standard_error=0.05, **kwargs):
        assert ax is not None, "Ax must be installed!"
        assert type(max_concurrent) is int and max_concurrent > 0
        self._ax = ax_client
        exp = self._ax.experiment
        self._objective_name = exp.optimization_config.objective.metric.name
        if self._ax._enforce_sequential_optimization:
            logger.warning("Detected sequential enforcement. Setting max "
                           "concurrency to 1.")
            max_concurrent = 1
        self._max_concurrent = max_concurrent
        self._parameters = list(exp.parameters)
        self._live_index_mapping = {}
        self._standard_error = standard_error
        self._num_completed = 0
        super(AxSearch2, self).__init__(**kwargs)

    def _suggest(self, trial_id):
        if self._num_live_trials() >= self._max_concurrent:
            return None
        parameters, trial_index = self._ax.get_next_trial()
        self._live_index_mapping[trial_id] = trial_index
        
        return parameters

    def on_trial_result(self, trial_id, result):
        pass

    def on_trial_complete(self,
                          trial_id,
                          result=None,
                          error=False,
                          early_terminated=False):
        """Notification for the completion of trial.

        Data of form key value dictionary of metric names and values.
        """
        if result:
            self._process_result(trial_id, result, early_terminated)
        self._live_index_mapping.pop(trial_id)

    def _process_result(self, trial_id, result, early_terminated=False):
        if early_terminated and self._use_early_stopped is False:
            return
        ax_trial_index = self._live_index_mapping[trial_id]
        metric_dict = {
            self._objective_name: (result[self._objective_name], self._standard_error*result[self._objective_name])
        }
        outcome_names = [
            oc.metric.name for oc in
            self._ax.experiment.optimization_config.outcome_constraints
        ]
        metric_dict.update({on: (result[on], self._standard_error*result[on]) for on in outcome_names})
        self._ax.complete_trial(
            trial_index=ax_trial_index, raw_data=metric_dict)

    def _num_live_trials(self):
        return len(self._live_index_mapping)

In [ ]:
init_notebook_plotting()

In [ ]:
ray.init(address='auto', redis_password='5241590000000000', log_to_driver=False)

In [ ]:
gs = GenerationStrategy(
    steps=[
        GenerationStep(
            model=Models.SOBOL,
            num_arms=25, 
            min_arms_observed=5, 
            recommended_max_parallelism=5, 
            enforce_num_arms=False, 
            model_kwargs={'deduplicate': True, 'seed': None},
            model_gen_kwargs=None
        ),
         GenerationStep(
            model=Models.GPEI,
            num_arms=-1,
            min_arms_observed=0,
            recommended_max_parallelism=20,
            enforce_num_arms=True, 
            model_kwargs=None, 
            model_gen_kwargs=None,
        )
    ]
)

In [ ]:
loss_sample_size = 50_000
data = pd.read_csv('data/credit_with_categoricals.csv')
data = data.dropna()
loss_sample_size = min(loss_sample_size, len(data))

axc = AxClient(generation_strategy=gs, verbose_logging=False, enforce_sequential_optimization=False)
axc.create_experiment(
    name="capacity_tuning",
    parameters=[
        {"name": "capacity", "type": "range", "bounds": [8, 128]},
        {"name": "latent_size", "type": "range", "bounds": [8, 128]},
        {"name": "num_layers", "type": "range", "bounds": [1, 4]},
        {"name": "residual_depths", "type": "range", "bounds": [2, 6]},
        {"name": "learning_rate", "type": "range", "bounds": [1e-5, 1e-1], "log_scale": True},
        {"name": "max_training_time", "type": "fixed", "value": 120.0}
    ],
    objective_name="mean_loss",
    minimize=True
)

In [ ]:
analysis = tune.run(
    train_evaluate,
    num_samples=100,
    search_alg=AxSearch2(axc, max_concurrent=20),  # Note that the argument here is the `AxClient`.
    verbose=1,  # Set this level to 1 to see status updates and to 2 to also see trial results.
    # To use GPU, specify: resources_per_trial={"gpu": 1}.
    resources_per_trial={"cpu": 2},
    max_failures=3,
    progress_reporter=JupyterNotebookReporter(overwrite=True, max_progress_rows=100)
)

In [ ]:
# Gets the best parameters from comparing all trials
params, mean_value = get_best_raw_objective_point(axc.experiment)
print("Best Trial Params:")
print(params, mean_value)

# Gets the parameters by predicting with the bayesian model
params, (mean_value, variance) = get_best_from_model_predictions(axc.experiment)
print("Estimated Best Params:")
print(params, mean_value, variance)



In [ ]:
render(axc.get_feature_importances())

In [ ]:
for param in params:
    try:
        render(plot_slice(axc.generation_strategy.model, param, 'mean_loss'))
    except ValueError:
        pass

In [ ]:
render(interact_contour(axc.generation_strategy.model, metric_name='mean_loss'))

In [ ]:
render(axc.get_optimization_trace())

In [ ]:
axc.get_trials_data_frame()